# API Request

In [1]:
import cdsapi
from pathlib import Path
import zipfile

In [ ]:
# ===============================
# CONFIGURATION – EASY TO MODIFY
# ===============================
years = range(2020, 2026)  # From 2020 to 2025 inclusive
months = range(1, 13)      # January to December
area = [71.5, -179.1, 18.9, -66.9]  # US bounding box: [N, W, S, E]
output_dir = Path.cwd() / "era5_downloads"
output_dir.mkdir(exist_ok=True)

variables = [
    "10m_u_component_of_wind",
    "10m_v_component_of_wind",
    "2m_dewpoint_temperature",
    "2m_temperature",
    "total_precipitation",
    "mean_sea_level_pressure",
    "surface_pressure"
]

# ===============================
# Download + Extract loop
# ===============================
client = cdsapi.Client()

for year in years:
    for month in months:
        year_str = str(year)
        month_str = str(month).zfill(2)
        days = [str(day).zfill(2) for day in range(1, 32)]  # CDS handles invalid days

        request = {
            "product_type": "reanalysis",
            "variable": variables,
            "year": year_str,
            "month": month_str,
            "day": days,
            "time": [f"{h:02d}:00" for h in range(24)],
            "data_format": "netcdf",
            "download_format": "zip",
            "area": area,
        }

        zip_filename = output_dir / f"era5_us_{year_str}_{month_str}.zip"
        print(f"Downloading ERA5 data for {year_str}-{month_str}...")

        try:
            client.retrieve("reanalysis-era5-single-levels", request, str(zip_filename))
            print(f"✔ Saved to {zip_filename}")

            # Extract only the data_stream-oper_stepType-instant.nc file
            with zipfile.ZipFile(zip_filename, 'r') as zip_ref:
                target_file = "data_stream-oper_stepType-instant.nc"
                if target_file in zip_ref.namelist():
                    print(f"Extracting {target_file}...")
                    zip_ref.extract(target_file, path=output_dir)
                    extracted_path = output_dir / target_file

                    # Rename extracted file
                    new_nc_filename = output_dir / f"era5_us_{year_str}_{month_str}.nc"
                    extracted_path.rename(new_nc_filename)
                    print(f"✔ Extracted and renamed to {new_nc_filename}")
                else:
                    print(f"⚠️ {target_file} not found in zip {zip_filename.name}")

            # Remove the zip file after extraction
            zip_filename.unlink()
            print(f"Deleted zip file {zip_filename.name}")

        except Exception as e:
            print(f"✘ Failed for {year_str}-{month_str}: {e}")


2025-06-03 17:08:53,025 INFO [2024-09-26T00:00:00] Watch our [Forum](https://forum.ecmwf.int/) for Announcements, news and other discussed topics.


2025-06-03 17:08:53,665 INFO Request ID is f4df2478-960b-4192-a484-a2390df9a750
2025-06-03 17:08:54,083 INFO status has been updated to accepted
2025-06-03 17:15:14,500 INFO status has been updated to successful


a18bda576afe6bc4a13aee4b1bd7a1e4.zip:   0%|          | 0.00/788M [00:00<?, ?B/s]

✔ Saved to c:\College\SUMMER 2025 SEMESTER\us_wildfire\data\raw_data\era5_downloads\era5_us_2020_01.zip
Extracting data_stream-oper_stepType-instant.nc...
✔ Extracted and renamed to c:\College\SUMMER 2025 SEMESTER\us_wildfire\data\raw_data\era5_downloads\era5_us_2020_01.nc
Deleted zip file era5_us_2020_01.zip


2025-06-03 17:18:22,329 INFO Request ID is 8f1c46c6-3353-4c9b-babd-472d3770b491
2025-06-03 17:18:22,471 INFO status has been updated to accepted
2025-06-03 17:18:44,211 INFO status has been updated to running
2025-06-03 17:24:42,813 INFO status has been updated to successful


...

2025-06-03 17:58:24,964 INFO Request ID is 10c35be2-da86-4074-aeaf-4528c12e366e
2025-06-03 17:58:25,325 INFO status has been updated to accepted
2025-06-03 17:58:34,376 INFO status has been updated to running
2025-06-03 18:04:46,376 INFO status has been updated to successful


13d5ec8c90bcd3ae57ab543f949c9e31.zip:   0%|          | 0.00/796M [00:00<?, ?B/s]

✔ Saved to c:\College\SUMMER 2025 SEMESTER\us_wildfire\data\raw_data\era5_downloads\era5_us_2020_05.zip
Extracting data_stream-oper_stepType-instant.nc...
✔ Extracted and renamed to c:\College\SUMMER 2025 SEMESTER\us_wildfire\data\raw_data\era5_downloads\era5_us_2020_05.nc
Deleted zip file era5_us_2020_05.zip


2025-06-03 18:30:45,141 INFO Request ID is 99d44e4e-7ec5-4c1f-aa59-bb5fa01ad378
2025-06-03 18:30:45,291 INFO status has been updated to accepted
2025-06-03 18:31:36,162 INFO status has been updated to running
2025-06-03 18:39:06,387 INFO status has been updated to successful


8950374b10649e1163ab3642ae8ab534.zip:   0%|          | 0.00/766M [00:00<?, ?B/s]